steps:

1. search for 'verilog' repos;
2. keep repo with permissive license;
3. download repo;

In [62]:
## 在python脚本里设置代理
import os
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [63]:
import time
from github import Github
import requests
import os
from tqdm import tqdm
from datetime import datetime, timedelta, date
import numpy as np
import pandas as pd
import codecs

from dotenv import load_dotenv
import  os
ACCESS_TOKEN = os.getenv('git_token')

In [64]:
g = Github(ACCESS_TOKEN)

In [65]:
g.get_rate_limit().core.raw_data

{'limit': 5000, 'used': 0, 'remaining': 5000, 'reset': 1712896820}

In [66]:
# stall requesting when reaching the limit (5000 requests per hour)
def pre_github_request_checker():
    rate_data = g.get_rate_limit().core.raw_data
    if rate_data['remaining'] < 50:
        time_to_reset = rate_data['reset'] - int(time.time()) + 1
        print(f"Sleeping for {time_to_reset} seconds")
        time.sleep(time_to_reset)

## Repository search

In [67]:
def search_github(language, start_date, end_date, permissible_lang_list=['Verilog', 'SystemVerilog']):
    """
    More info:
    https://docs.github.com/en/search-github/getting-started-with-searching-on-github/understanding-the-search-syntax#query-for-dates
    """
    assert language in permissible_lang_list
    date_q = f"{start_date.strftime('%Y-%m-%d')}..{end_date.strftime('%Y-%m-%d')}" 
    result = g.search_repositories("",language=language, created=date_q) 
    print(f"Found {result.totalCount} repos for: {language}, {date_q}")
    return result

# results = search_github('Verilog', datetime(1980,1,1), datetime(2010,1,1))
# results = search_github('Verilog', datetime(1980,1,1), datetime(2015,1,1))

In [68]:
def add_repo_to_df(df, repo, data_field):
    data = [getattr(repo, attr) for attr in data_field]
    data.append(repo.get_topics())
    try:
        license_url = repo.get_license().license.url
    except:
        license_url = "None"
    data.append(license_url)
    df.loc[len(df)] = data
    return df

In [69]:
# get repo info, add to df
def process_repo_search_results(df, results, data_field):
    for i in range(1000):
        rate_data = g.get_rate_limit().core.raw_data
        now_seconds = int(time.time())
        if rate_data['remaining'] < 100:
            time_to_reset = rate_data['reset'] - int(time.time()) + 1
            print(f"Sleeping for {time_to_reset} seconds")
            time.sleep(time_to_reset)
        page = results.get_page(i)
        page_size = len(page)
        for j in range(page_size):
            df = add_repo_to_df(df, page[j], data_field)
        if page_size < 30:
            break   
    return df

In [80]:
from os import path as osp

def init_df(data_field):
    return pd.DataFrame(columns=data_field + ["topics","license_url"])

def make_datetime(string, format='%Y-%m-%d'):
    return datetime.strptime(string, format)

def str_datetime(dt, format='%Y-%m-%d'):
    return dt.strftime(format)

def append_msg(msg, path):
    with open(path, 'a') as fp:
        fp.write(msg+'\n')

def find_repos(language, date_list, data_field, root_path='data/repo_index', search_limit=1000):
    os.makedirs(root_path, exist_ok=True)
    log_path = osp.join(root_path, 'log')
    
    while len(date_list)>0:
        start_date, end_date = date_list.pop()
        # search by lang, start & end dates
        repo_search_results = search_github(language, start_date, end_date)
        if repo_search_results.totalCount > 0:
            if repo_search_results.totalCount >= search_limit:
                # Reduce date range recursively
                delta = (end_date - start_date) / 2
                date_list.append((start_date + delta, end_date))
                date_list.append((start_date, end_date - delta))
                print(f"Search count > {search_limit} ({repo_search_results.totalCount}) in ({start_date}, {end_date}), split to {date_list[-1]} and {date_list[-2]}")
            else:
                df = init_df(data_field)
                process_repo_search_results(df, repo_search_results, data_field)
                file_name = f"{language}_{str_datetime(start_date)}_{str_datetime(end_date)}.csv"
                df.to_csv(osp.join(root_path, file_name))
                msg = f"Done: {str_datetime(start_date)}..{str_datetime(end_date)}, length: {len(df)}"
                append_msg(msg, log_path)
                print(msg)

In [82]:
data_field = [
    "id",
    "clone_url",
    "created_at",
    "description",
    "full_name",
    "language",
    "name",
    "size",
    "stargazers_count",
    "updated_at",
    "forks_count"
]
data_field.sort()
language = "Verilog"

# find_repos(language, [(datetime(2010,1,1), datetime(2010,5,1))], data_field, search_limit=10)

## add date_range to task list

since github api's max rate limit per hour is 1000, so we need to split the search date range. you can either:

1. set the whole range and let the function find_repo to do the binary split for you;
2. or split manually (likely based on previous search experience) and pass the date_list to find_repo;

1. set full range

In [94]:
start_date = datetime(2000,1,1)
end_date = datetime(2023,11,7)
date_list = [(start_date, end_date)]

2. set list of ranges

In [83]:
import re

def find_num_date(text):
    # Extracting number and dates using one pattern
    pattern = r'Found (\d+) repos for: \w+, (\d{4}-\d{2}-\d{2})\.\.(\d{4}-\d{2}-\d{2})'
    match = re.search(pattern, text)
    if match:
        number = match.group(1)
        start_date = match.group(2)
        end_date = match.group(3)
        # print("Number:", number)
        # print("Start Date:", start_date)
        # print("End Date:", end_date)
        return (number, (start_date, end_date))
    else:
        # print("No match found.")
        return None

In [84]:
with open('github.log') as fp:
    log = fp.readlines()
for i in range(10):
    print(log[i], end="")

Empty DataFrame
Columns: [clone_url, created_at, description, forks_count, full_name, id, language, name, size, stargazers_count, updated_at, topics, license_url]
Index: []
Found 1000 repos for: Verilog, 2000-01-01..2024-04-10
Found 279 repos for: Verilog, 2000-01-01..2012-02-20
Done: 2000-01-01..2012-02-20, df length: 279
Found 1000 repos for: Verilog, 2012-02-20..2024-04-10
Found 1000 repos for: Verilog, 2012-02-20..2018-03-17
Found 1000 repos for: Verilog, 2012-02-20..2015-03-04
Found 643 repos for: Verilog, 2012-02-20..2013-08-27


In [87]:
from pprint import pprint
date_list = []
for l in log:
    ret = find_num_date(l)
    if ret is not None:
        if eval(ret[0]) != 1000:
            date_list.append(
                (make_datetime(ret[1][0]), make_datetime(ret[1][1]))
                )
            
# reverse the list and pop from back
date_list = date_list[::-1]
pprint(date_list[-5:])
print(len(date_list))

[(datetime.datetime(2014, 10, 17, 0, 0), datetime.datetime(2015, 3, 4, 0, 0)),
 (datetime.datetime(2014, 5, 31, 0, 0), datetime.datetime(2014, 10, 17, 0, 0)),
 (datetime.datetime(2013, 8, 27, 0, 0), datetime.datetime(2014, 5, 31, 0, 0)),
 (datetime.datetime(2012, 2, 20, 0, 0), datetime.datetime(2013, 8, 27, 0, 0)),
 (datetime.datetime(2000, 1, 1, 0, 0), datetime.datetime(2012, 2, 20, 0, 0))]
92


In [91]:
find_repos(language, date_list, data_field, root_path='data/repo_index', search_limit=1000)

Found 1000 repos for: Verilog, 2023-11-07..2024-04-13
Search count > 1000 (1000) in (2023-11-07 00:00:00, 2024-04-13 19:46:51.460603), split to (datetime.datetime(2024, 1, 25, 9, 53, 25, 730302), datetime.datetime(2024, 4, 13, 19, 46, 51, 460603)) and (datetime.datetime(2023, 11, 7, 0, 0), datetime.datetime(2024, 1, 25, 9, 53, 25, 730301))
Found 1000 repos for: Verilog, 2024-01-25..2024-04-13
Search count > 1000 (1000) in (2024-01-25 09:53:25.730302, 2024-04-13 19:46:51.460603), split to (datetime.datetime(2024, 3, 5, 2, 50, 8, 595452), datetime.datetime(2024, 4, 13, 19, 46, 51, 460603)) and (datetime.datetime(2024, 1, 25, 9, 53, 25, 730302), datetime.datetime(2024, 3, 5, 2, 50, 8, 595453))
Found 1000 repos for: Verilog, 2024-03-05..2024-04-13
Search count > 1000 (1000) in (2024-03-05 02:50:08.595452, 2024-04-13 19:46:51.460603), split to (datetime.datetime(2024, 3, 24, 23, 18, 30, 28028), datetime.datetime(2024, 4, 13, 19, 46, 51, 460603)) and (datetime.datetime(2024, 3, 5, 2, 50, 8, 

In [95]:
language = 'SystemVerilog'
find_repos(language, date_list, data_field, root_path='data/repo_index', search_limit=1000)

Found 1000 repos for: SystemVerilog, 2000-01-01..2023-11-07
Search count > 1000 (1000) in (2000-01-01 00:00:00, 2023-11-07 00:00:00), split to (datetime.datetime(2011, 12, 4, 12, 0), datetime.datetime(2023, 11, 7, 0, 0)) and (datetime.datetime(2000, 1, 1, 0, 0), datetime.datetime(2011, 12, 4, 12, 0))
Found 1000 repos for: SystemVerilog, 2011-12-04..2023-11-07
Search count > 1000 (1000) in (2011-12-04 12:00:00, 2023-11-07 00:00:00), split to (datetime.datetime(2017, 11, 20, 6, 0), datetime.datetime(2023, 11, 7, 0, 0)) and (datetime.datetime(2011, 12, 4, 12, 0), datetime.datetime(2017, 11, 20, 6, 0))
Found 1000 repos for: SystemVerilog, 2017-11-20..2023-11-07
Search count > 1000 (1000) in (2017-11-20 06:00:00, 2023-11-07 00:00:00), split to (datetime.datetime(2020, 11, 13, 3, 0), datetime.datetime(2023, 11, 7, 0, 0)) and (datetime.datetime(2017, 11, 20, 6, 0), datetime.datetime(2020, 11, 13, 3, 0))
Found 1000 repos for: SystemVerilog, 2020-11-13..2023-11-07
Search count > 1000 (1000) in 

In [ ]:
len(df2)

4

In [ ]:
df2.to_csv(f"data/{language}_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}.csv")

# concate repo dfs and deduplicate

In [33]:
def combine_and_deduplicate_gh_search_results(csvs):
    df = pd.concat(map(lambda x: pd.read_csv(x, na_values=['None']), csvs), ignore_index=True)
    df = df.drop(['Unnamed: 0'],axis=1)
    df = df.drop_duplicates([c for c in df.columns if c != 'updated_at'])
    return df

In [34]:
from os import path as osp
import os

v_filepaths = ['./Verilog_1980-01-01_2012-01-01.csv']
sv_filepaths = ['./SystemVerilog_1980-01-01_2012-01-01.csv']

verilog_df = combine_and_deduplicate_gh_search_results(v_filepaths)
systemverilog_df = combine_and_deduplicate_gh_search_results(sv_filepaths)

print(len(verilog_df))
print(len(systemverilog_df))

repo_indices_dir = 'data/search_repo_indices'
os.makedirs(repo_indices_dir)
verilog_df.to_csv(osp.join(repo_indices_dir,"full_verilog_repos.csv"))
systemverilog_df.to_csv(osp.join(repo_indices_dir,"full_systemverilog_repos.csv"))

237
4


In [ ]:
all_df = pd.concat([verilog_df,systemverilog_df]).drop_duplicates(subset=[c for c in verilog_df.columns if not c in ['language']])
all_df.to_csv("data/all_deduplicated_repos.csv")
print(len(all_df))

### define extensions to keep

In [ ]:
# https://www.intel.com/content/www/us/en/programmable/quartushelp/17.0/reference/glossary/glosslist.htm
# https://marketplace.visualstudio.com/items?itemName=eirikpre.systemverilog
verilog_extension_files = ['v','verilog','vlg','vh']
system_verilog_extension_files = ['sv','svh','svp', 'sva']
extra_file_types = ['vo','vt'] # verilog output, verilog test bench

extensions_to_keep = verilog_extension_files + system_verilog_extension_files + extra_file_types

## Finding licenses and remove non permissive repo

In [37]:
def add_licenses_from_repo_df_to_dict(repo_df, licenses_dict):
    df_with_unique_licenses = repo_df.loc[repo_df['license_url'].dropna().drop_duplicates().index]
    repo_ids = list(df_with_unique_licenses['id'])
    for rid in repo_ids:
        pre_github_request_checker()
        license_data = g.get_repo(rid).get_license().license.raw_data
        licenses_dict[license_data['url']] = license_data

In [38]:
licenses_dict = {}
add_licenses_from_repo_df_to_dict(verilog_df,licenses_dict)
add_licenses_from_repo_df_to_dict(systemverilog_df,licenses_dict)
df = pd.DataFrame.from_dict(licenses_dict, orient='index')
df.to_csv(os.path.join('data/search_repo_indices','licenses.csv'))

In [39]:
def get_licenses_with_permissions_conditions(license_df,permissions=[],conditions=[]):
    indices = []
    for i, row in license_df.iterrows():
        if len(permissions) == 0 or set(permissions).issubset(set(row['permissions'])):
            if len(conditions) == 0 or len(set(conditions).intersection(set(row['conditions']))) > 0 :
                indices.append(i)
    return license_df.loc[indices]

In [40]:
permissions = ['modifications','distribution']
special_conditions = ['same-license--file','same-license--library','same-license']
permissive_licenses_df = get_licenses_with_permissions_conditions(df,permissions=permissions,conditions=[])
distributive_licenses_df = get_licenses_with_permissions_conditions(df,permissions=['distribution'],conditions=[])

In [41]:
print(len(permissive_licenses_df))
print(len(distributive_licenses_df))

8
8


Conclusion, all licenses found are permissive in that they allow modifications and distribution! Repos without licenses are not included

In [42]:
p_sv_df = systemverilog_df.dropna(subset=['license_url'])
p_ve_df = verilog_df.dropna(subset=['license_url'])

p_sv_df.to_csv(os.path.join(repo_indices_dir,"permissive_systemverilog_repos.csv"))
p_ve_df.to_csv(os.path.join(repo_indices_dir,"permissive_verilog_repos.csv"))

In [44]:
p_all_df = pd.concat([p_ve_df,p_sv_df]).drop_duplicates(subset=[c for c in verilog_df.columns if not c in ['language']])
p_all_df.to_csv("data/permissive_all_deduplicated_repos.csv")
print(len(p_all_df))

In [89]:
len(df)
all_permissions = []
all_conditions = []
all_limitations = []
for i,row in df.iterrows():
    all_permissions.append(row['permissions'])
    all_conditions.append(row['conditions'])
    all_limitations.append(row['limitations'])

In [72]:
all_conditions

[['include-copyright', 'document-changes', 'disclose-source', 'same-license'],
 ['include-copyright', 'document-changes'],
 ['include-copyright', 'document-changes', 'disclose-source', 'same-license'],
 ['include-copyright'],
 ['include-copyright'],
 ['include-copyright',
  'disclose-source',
  'document-changes',
  'same-license--library'],
 ['include-copyright',
  'disclose-source',
  'document-changes',
  'same-license--library'],
 [],
 ['include-copyright'],
 ['include-copyright',
  'document-changes',
  'disclose-source',
  'network-use-disclose',
  'same-license'],
 [],
 ['include-copyright'],
 [],
 ['disclose-source', 'include-copyright', 'same-license--file'],
 ['include-copyright', 'document-changes'],
 ['include-copyright', 'document-changes', 'same-license'],
 ['disclose-source', 'include-copyright', 'same-license'],
 ['include-copyright--source'],
 ['include-copyright--source', 'document-changes'],
 ['include-copyright', 'document-changes'],
 ['disclose-source', 'include-co

## download repos:

There are two ways:
1. Clone or download full repo, then filter out files;
2. get into repos and download single files if its extension is the target ones.

The first method requires more storage space and network usage, but the second method requires more github api requests (for each file, it requires one get_repo search + n get_contents search).

therefore, the first method is prefered

### 1. raw git clone

1.1 create downloading bash script

In [51]:
def create_clone_command(clone_url, directory, depth=1, branch='master'):
    command_parts = [f'git clone']
    # TODO: rerun search without master branch (causes error for some searches!)
    # if branch is not set, then the clone method will download default branch
    # command_parts.append(f"-b {branch}") 
    command_parts.append(f"--depth {depth}")
    command_parts.append(f"--no-tags")
    # command_parts.append(f"--no-checkout")
    command_parts.append(clone_url)
    command_parts.append(directory)
    return " ".join(command_parts)

def create_clone_script_for_df(repo_df,script_out_path,clone_out_dir):
    with open(script_out_path,'w+') as f:
        for i,row in repo_df.iterrows():
            clone_url = row['clone_url']
            out_dir = os.path.join(clone_out_dir,str(row['id']))
            if not os.path.exists(out_dir):
                os.mkdir(out_dir)
            f.write(create_clone_command(clone_url,str(os.path.abspath(out_dir))).replace("\\","/") + "\n")

create_clone_command("https://github.com/mrehkopf/sd2snes.git","./data/full_repos/exp")

'git clone --depth 1 --no-tags https://github.com/mrehkopf/sd2snes.git ./data/full_repos/exp'

In [50]:
os.makedirs("data/full_repos/permissive", exist_ok=True)
create_clone_script_for_df(p_all_df, "./data/clone_all_p.sh","data/full_repos/permissive")

In [84]:
# !bash data/clone_all_p.sh

1.2. Filter repos

In [ ]:
# argument '1' in rsplit can limit the split to max 2 segments
def get_file_extension(path):
    return path.rsplit(".",1)[-1]

In [58]:
def delete_all_files_without_right_extension(start_dir, extensions_to_keep):
    errors = []
    for root, dirs, files in os.walk(start_dir):
        for file in [sf for sf in files if not get_file_extension(sf) in extensions_to_keep]:
            file_path = os.path.join(root,file)
            try:
                os.remove(file_path)
            except Exception as e:
                errors.append(e)
    return errors

def delete_empty_dirs(start_dir):
    errors = []
    for root, dirs, files in os.walk(start_dir,topdown=False):
        for d in dirs:
            dir_path = os.path.join(root,d)
            if len(os.listdir(dir_path)) == 0:
                try:
                    os.rmdir(dir_path)
                except Exception as e:
                    errors.append(e)
    return errors

In [59]:
start_dir = "data/full_repos/permissive"

files_errors = delete_all_files_without_right_extension(start_dir,extensions_to_keep)
dirs_errors = delete_empty_dirs(start_dir)

### 2. fine-grained repo download

In [1]:
def update_files_dict(count, content):
    global files_dict
    files_dict[count] = {
        "path": content.raw_data['path'],
        "size": content.raw_data['size'],
        "count_id": count
    }

def download_content(repo,content,extensions,out_dir):
    content_raw_data = content.raw_data
    content_type = content_raw_data['type']
    if content_type == 'dir':
        pre_github_request_checker()
        new_contents = repo.get_contents(content_raw_data['path'])
        for new_content in new_contents:
            download_content(repo,new_content,extensions,out_dir)
    elif content_type == 'file':
        extension = get_file_extension(content_raw_data['name'])
        if extension in extensions:
            global file_count
            update_files_dict(file_count,content)
            pre_github_request_checker()
            try:
                with open(os.path.join(out_dir,str(file_count) + "." + extension),'wb') as f:
                    f.write(content.decoded_content)
                file_count += 1
            except Exception as e:
                print(f"Caught exception while trying to write content:\n{e}")
    # raise Exception(f"Content type not recognized: {content_type}")

def download_files_from_repo(repo,extensions,out_dir):
    global files_dict
    global file_count
    file_count = 0
    files_dict = {}
    pre_github_request_checker()
    # get file list at root path
    contents = repo.get_contents("/")
    for content in contents:
        download_content(repo,content,extensions,out_dir)
    df = pd.DataFrame.from_dict(files_dict,orient='index')
    print(f"Saving csv index with {len(df)} entries")
    df.to_csv(os.path.join(out_dir,"index.csv"))
    
def download_all_repos(df,extensions,out_dir):
    all_repo_ids = list(df['id'])
    for i in range(0,len(df['id'])):
        repo_id = all_repo_ids[i]
        pre_github_request_checker()
        repo = g.get_repo(repo_id)
        repo_dir = os.path.join(out_dir,str(repo_id))
        if not os.path.exists(repo_dir):
            os.makedirs(repo_dir)
        print(f"Searching repo {i} with id: {repo_id}")
        download_files_from_repo(repo,extensions,repo_dir)

In [23]:
# g.rate_limiting
# repo = g.get_repo(279998)
# contents = repo.get_contents("/")
# print(contents)
# contents[0].raw_data

In [55]:
download_all_repos(systemverilog_df, verilog_extension_files, "./data/full_repos/raw")

NameError: name 'verilog_extension_files' is not defined

## Create index file

This step collects all files into data samples, each file is a code file, which is followed by the dataset splitting step.

In [10]:
def create_files_df(downloaded_repo_dir,extensions_to_keep):
    extensions_map = {ext: True for ext in extensions_to_keep}
    df = pd.DataFrame(columns=['directory','repo_id','file_name','extension'])
    for repo_id in os.listdir(downloaded_repo_dir):
        for root,dirs,files in os.walk(os.path.join(downloaded_repo_dir,repo_id)):
            for file in files:
                extension = get_file_extension(file)
                try:
                    if extensions_map[extension]:
                        directory = os.path.join(root,file)
                        df.loc[len(df)] = [directory, repo_id, file, extension]
                except Exception as e:
                    print(f"Error: {e}")
                    extensions_map[extension] = False
        print(f"Done with repo: {repo_id}")
    print(f"Extensions: {extensions_map}")
    return df

In [ ]:
verilog_files_df = create_files_df('data/full_repos/permissive',verilog_extension_files + system_verilog_extension_files)

In [13]:
verilog_files_df.to_csv('./files_index.csv')

## Partition dataset for processing

In [169]:
files_index = pd.read_csv('data/search_repo_indices/files_index.csv',index_col=0)
# files_index

In [170]:
files_index = files_index[files_index['extension'].isin(extensions_to_keep)]
# files_index = files_index.reset_index(drop=True)
len(files_index)

314877

In [171]:
files_index = files_index.reset_index(drop=True)
# files_index

In [173]:
few_indices = np.random.choice(len(files_index),replace=False,size=200)
remaining_files_index = files_index.drop(index=few_indices)

In [174]:
len(remaining_files_index)

314677

In [175]:
number_of_partitions = 10
tot_len = len(remaining_files_index)
for i in range(number_of_partitions):
    partition_df = remaining_files_index.iloc[list(range(i*tot_len//number_of_partitions,(i+1)*tot_len//number_of_partitions))]
    partition_df.to_csv(f"data/verilog_partitions/files_index_part_{i}.csv")

In [176]:
tot_len = len(remaining_files_index)
total = 0
for i in range(number_of_partitions):
    length = len(pd.read_csv(f"data/verilog_partitions/files_index_part_{i}.csv"))
    total += length

## Fill partitions with source code

In [189]:
def read_source_code(directory):
    # Biggest error = utf-8 encoding problem
    try:
        # return open(directory,'r').read()
        with codecs.open(directory,encoding='utf-8', errors='replace', mode = 'r') as f:
            data = f.read()
        return data.replace("\x00","") # replacing this might not be needed but someone online said it helps...
    except Exception as e:
        e_string = f"0:FOUND ERROR: {e}"
        print(e_string)
        return e_string

def clean_row_directory(row):
    return row['directory'].replace("\\","/")

def add_source_code_to_index_df(df):
    df['directory'] = df.apply(lambda row: clean_row_directory(row),axis=1)
    df['code'] = ""
    tqdm.pandas(desc='Apply read_source_code')
    df['code'] = df.progress_apply(lambda row: read_source_code(row['directory']),axis=1)
    return df

In [190]:
number_of_partitions = 10
for i in range(number_of_partitions):
    df_dir = f"data/verilog_partitions/files_index_part_{i}.csv"
    print(f"Starting {i}")
    partition_df = pd.read_csv(df_dir,index_col=0)
    new_partition_df = add_source_code_to_index_df(partition_df)
    new_partition_df.to_csv(df_dir)
    del partition_df, new_partition_df
print("All done!")

Starting 9


Apply read_source_code:  94%|█████████▎| 29496/31468 [05:13<00:28, 70.05it/s] 

e_string


Apply read_source_code: 100%|██████████| 31468/31468 [05:48<00:00, 90.35it/s]


All done!
